In [ ]:
#using python-docx instead of spire.doc (pls work)


#alright scratch everything, you are basically going to web-scrape all the documents and put the info into the spreadsheet
    #2:10 pm - work on the none's - there are originally 9 of em 

    #2:24 pm - if i change the end word to "Materials Disposition" different files return None (7 of em)

    #2:50 pm - ok we got all the info except for from the pdfs. Now we have to figure out how to get the other info in the spread sheet

#4:02 pm Aug 28: code finally works and i have to add the other columns. But, I'm looking back on my code and now I have no idea how it works

In [732]:
#to go through all the docs in a folder
import os
#for pdf
from pdfminer.high_level import extract_text
#get the stuff from the header
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer
#to add them to an excel sheet
import openpyxl

summarized_list = []
def files_iterator(directory):
    keys = {'RGA#', 'Cust No', 'SO', 'PO', 'Inv#', 'C/NC/CC', 'Desc'}
    out_text = {key: '' for key in keys}

    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        out_text['RGA#'] = filename
        #Getting the stuff from the header first
        header_list = header_lister(file_path)

        #The function header_lister gives us a list of all the info we need from the header (if provided),
        #however they are not all in the same order (but there is still some consistency within the lists)
        #so we will separate them into groups based off organization then
        #add them to summarized_list based off of that (bc I see no faster option)
        
    #first group is if the title is shown after the value
            #Cust No
        if header_list.index('CUST NO') == 1:
            out_text['Cust No'] = header_list[header_list.index('CUST NO') - 1]
            #SO
            if header_list[header_list.index('SO')-1] == 'CUST NO':
                out_text['SO'] = 'None'
            else:
                out_text['SO'] = header_list[header_list.index('SO')-1]
            #PO
            if header_list[header_list.index('PO')-1] == 'SO':
                out_text['PO'] = 'None'
            else:
                out_text['PO'] = header_list[header_list.index('PO')-1]
            # Inv#
            if header_list[header_list.index('INV #') - 1] == 'PO':
                out_text['Inv#'] = 'None'
            else:
                if header_list[header_list.index('INV #')+1] in ('C', 'CC', 'NC'):
                    out_text['Inv#'] = header_list[header_list.index('INV #')-1]
                else:
                    out_text['Inv#'] = (header_list[header_list.index('INV #')-1] +' '+ header_list[header_list.index('INV #')+1])
            #C/NC/CC
            if header_list[header_list.index('C/NC/CC')-1] in ('C', 'CC', 'NC'):
                out_text['C/NC/CC'] = header_list[header_list.index('C/NC/CC')-1]
            else:
                out_text['C/NC/CC'] = 'None'
    #second group is for if the title is shown before the value
            #Cust No
        if header_list.index('CUST NO') == 0:
            out_text['Cust No'] = header_list[1]
            #SO
            if header_list[header_list.index('SO')+1] == 'PO':
                out_text['SO'] = 'None'
            else:
                out_text['SO'] = header_list[header_list.index('SO')+1]
            #PO
            if header_list[header_list.index('PO')+1] == 'INV #':
                out_text['PO'] = 'None'
            else:
                out_text['PO'] = header_list[header_list.index('PO')+1]
            # Inv#
            if header_list[header_list.index('INV #')+1] not in ('C', 'CC', 'NC'):
                if header_list[header_list.index('INV #')+2] not in ('C', 'CC', 'NC'):
                    if header_list[header_list.index('INV #')+3] not in ('C', 'CC', 'NC'):
                        out_text['Inv#'] = (header_list[header_list.index('INV #')+1] +' '+ header_list[header_list.index('INV #')+2] +' '+ header_list[header_list.index('INV #')+3])
                    else:
                        out_text['Inv#'] = (header_list[header_list.index('INV #')+1] +' '+ header_list[header_list.index('INV #')+2])
                else:
                    out_text['Inv#'] = header_list[header_list.index('INV #')+1]
            #C/NC/CC
            if header_list[header_list.index('C/NC/CC')+1] == None:
                out_text['C/NC/CC'] = None
            else:
                out_text['C/NC/CC'] = header_list[header_list.index('C/NC/CC')+1]
        
    #The next portion focuses on getting the description
        extracted_text = extract_text(file_path, page_numbers=[0])
        #lines = text.split('\n')  # Split the text into lines
        start_word = 'Customer Problem Description: '
        end_word = 'Material Disposition: '
        start_index = extracted_text.find(start_word)
        end_index = extracted_text.find(end_word, start_index)
        if start_index != -1 and end_index != -1:
            extracted_text = extracted_text[start_index + len(start_word):end_index].strip()
        if start_index != -1 and end_index == -1:
            end_word2 = 'Power Transmission Solutions'
            end_index2 = extracted_text.find(end_word2, start_index)
            extracted_text = extracted_text[start_index + len(start_word):end_index2].strip()
        phrase = "The returned item was inspected"
        phrase_plural = "The returned items were inspected"
        phrase2 = "no manufacturing anomalies"
        if phrase2 in extracted_text:
            extracted_text = extracted_text.split(phrase)[0]
            extracted_text = extracted_text.split(phrase_plural)[0]
        out_text['Desc'] = extracted_text

        summarized_list.append(out_text.copy())
#The code below adds all components of out_texts to a spreadsheet
    wb = openpyxl.load_workbook("Pranav_RGA_Reports.xlsx")
    sheet = wb.active
    column_names = ['RGA#', 'Cust No', 'SO', 'PO', 'Inv#','C/NC/CC', 'Description']
    for col_num, column_title in enumerate(column_names, 1):
        sheet.cell(row=1, column=col_num).value = column_title
    
    row_count = 2
    for dictionary in summarized_list:
        sheet.cell(row=row_count, column=1).value = dictionary['RGA#']
        sheet.cell(row=row_count, column=2).value = dictionary['Cust No']
        sheet.cell(row=row_count, column=3).value = dictionary['SO']
        sheet.cell(row=row_count, column=4).value = dictionary['PO']
        sheet.cell(row=row_count, column=5).value = dictionary['Inv#']
        sheet.cell(row=row_count, column=6).value = dictionary['C/NC/CC']
        sheet.cell(row=row_count, column=7).value = dictionary['Desc']

        row_count += 1
    wb.save("Pranav_RGA_Reports.xlsx")
        

def header_lister(pdf_path, page_number = 1):
    header_text = []
    for page_layout in extract_pages(pdf_path):
        if page_layout.pageid == page_number:
            text_elements = []
            for element in page_layout:
                if isinstance(element, LTTextContainer):
                    # Check if the element is in the header (top of the page)
                    if element.y1 > page_layout.height - 100:
                        for line in element:
                            if isinstance(line, LTTextContainer):
                                # Check if the line is in the right half of the page
                                if line.x0 > page_layout.width / 2:
                                    text_elements.append(line)
            
            # Sort text elements by y-coordinate (top to bottom)
            text_elements.sort(key=lambda e: -e.y1)
            
            for line in text_elements:
                header_text.append(line.get_text().strip())
    
    return header_text



directory = r"Docs 2 pdfs"
files_iterator(directory)
        
    

In [ ]:
#to go through all the docs in a folder
import os
#to extract certain text from the doc
from docx import Document
#for pdf
from pdfminer.high_level import extract_text
#get the stuff from the header
#from docx2python import docx2python
#take out the unnecessary spaces
import re
#to add them to an excel sheet
import openpyxl

summarized_list = []
def files_iterator(directory):
    keys = {'RGA#', 'Cust No', 'SO', 'PO', 'Inv#', 'Desc'}

    #got each filename in here
    for filename in os.listdir(directory):
        out_text = {key: '' for key in keys}
        file_path = os.path.join(directory, filename)
        out_text['RGA#'] = filename
        

        if filename.endswith('docx'):
            #got and cleaned up the extracted text here
            extracted_text = Extract_text(file_path)
            #Eliminates the extra spaces that are read by "extract_text function" bc of the line breaks on the actual paper
            extracted_text =" ".join(extracted_text.split())
        
        if filename.endswith('pdf'):
            #this is for stuff in the header, only for pdfs though
            text = extract_text(file_path, page_numbers=[0])
            lines = text.split('\n')  # Split the text into lines
            filtered_lines = []

            for line in lines:
                if 'CUST NO ' in line or 'CUST NO ' in filtered_lines:
                    filtered_lines.append(line)
            filtered_lines = [item for item in filtered_lines if item != '']
            filtered_lines = filtered_lines[5:10]
            out_text['Cust No'] = filtered_lines[0]
            out_text['SO'] = filtered_lines[1]
            out_text['PO'] = filtered_lines[2]
            out_text['Inv#'] = filtered_lines[3]
            
            #this line does not call the function i made. It is a part of the library for pdfminer. That is why I name my function
            #with a capital E
            extracted_text = extract_text(file_path)
            start_word = 'Customer Problem Description: '
            end_word = 'Material Disposition: '
            start_index = extracted_text.find(start_word)
            end_index = extracted_text.find(end_word, start_index)
            if start_index != -1 and end_index != -1:
                extracted_text = extracted_text[start_index + len(start_word):end_index].strip()
            if start_index != -1 and end_index == -1:
                end_word2 = 'Power Transmission Solutions'
                end_index2 = extracted_text.find(end_word2, start_index)
                extracted_text = extracted_text[start_index + len(start_word):end_index2].strip()
        
        phrase = "The returned item was inspected"
        phrase_plural = "The returned items were inspected"
        phrase2 = "no manufacturing anomalies"
        if phrase2 in extracted_text:
            extracted_text = extracted_text.split(phrase)[0]
            extracted_text = extracted_text.split(phrase_plural)[0]
        out_text['Desc'] = extracted_text
        

        summarized_list.append(out_text.copy())
    
    wb = openpyxl.load_workbook("Pranav_RGA_Reports.xlsx")
    sheet = wb.active
    column_names = ['RGA#', 'Cust No', 'SO', 'PO', 'Inv#', 'Description']
    for col_num, column_title in enumerate(column_names, 1):
        sheet.cell(row=1, column=col_num).value = column_title
    #Iterate over the items in out_text and add them to the Excel sheet

    row_count = 2
    for dictionary in summarized_list:
        sheet.cell(row=row_count, column=1).value = dictionary['RGA#']
        sheet.cell(row=row_count, column=2).value = dictionary['Cust No']
        sheet.cell(row=row_count, column=3).value = dictionary['SO']
        sheet.cell(row=row_count, column=4).value = dictionary['PO']
        sheet.cell(row=row_count, column=5).value = dictionary['Inv#']
        sheet.cell(row=row_count, column=6).value = dictionary['Desc']

        row_count += 1
    wb.save("Pranav_RGA_Reports.xlsx")


def Extract_text(doc_path):
    start_word = 'Customer Problem Description:'
    end_word = 'Material Disposition'
    end_word2 = "Analysis"

    doc = Document(doc_path)
    full_text = []

    for para in doc.paragraphs:
        full_text.append(para.text)
    full_text = ' '.join(full_text)

    start_index = full_text.find(start_word)
    end_index = full_text.find(end_word, start_index)

    if start_index == -1:
        return "None"

    if end_index == -1:
        end_index = full_text.find(end_word2, start_index)
    
    return full_text[start_index + len(start_word):end_index].strip()


directory = r"Docs 2 pdfs"
files_iterator(directory)


In [727]:
from pdfminer.high_level import extract_text
import os

def ugh(directory):
    keys = {'RGA#', 'Cust No', 'SO', 'PO', 'Inv#', 'C/NC/CC', 'Desc'}
    for filename in os.listdir(directory):
        out_text = {key: '' for key in keys}
        file_path = os.path.join(directory, filename)
        extracted_text = extract_text(file_path, page_numbers=[0])
        #lines = text.split('\n')  # Split the text into lines
        start_word = 'Customer Problem Description: '
        end_word = 'Material Disposition: '
        start_index = extracted_text.find(start_word)
        end_index = extracted_text.find(end_word, start_index)
        if start_index != -1 and end_index != -1:
            extracted_text = extracted_text[start_index + len(start_word):end_index].strip()
        if start_index != -1 and end_index == -1:
            end_word2 = 'Power Transmission Solutions'
            end_index2 = extracted_text.find(end_word2, start_index)
            extracted_text = extracted_text[start_index + len(start_word):end_index2].strip()
        phrase = "The returned item was inspected"
        phrase_plural = "The returned items were inspected"
        phrase2 = "no manufacturing anomalies"
        if phrase2 in extracted_text:
            extracted_text = extracted_text.split(phrase)[0]
            extracted_text = extracted_text.split(phrase_plural)[0]
        print(filename)
        print(extracted_text)
        
directory = 'Docs 2 pdfs'
ugh(directory)




5366138.pdf
SB 22226 W33 SS: Making Noise 

It was reported this unit was used on a paper cup manufacturing machine that supported a 
radial load exposed to a clean environment.  After installation, the bearing operated for a short 
period when it was removed for noise issues. 

REGAL application engineering received the part that was disassembled for component 
inspection.  Its inner and outer ring raceways showed axial marks also known as false brinelling.  
These damages were located at various locations and may have occurred from improper 
installation, bearing removal or external vibration while the bearing was stationary.  Subsequent 
bearing operation can then have the rolling elements running overs these marks and could then 
produce the noise as observed by the customer. 

At this time, courtesy credit will be issued.  Proper installation should be followed as detailed on 
the instruction manual.  In addition, the machine should be reviewed to ensure vibration was 
reduced whi

In [690]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextBoxHorizontal, LTTextLineHorizontal, LTTextContainer

# def header_lister(pdf_path, header_height=100):
#     header_text = []
    
#     # Extract the layout of the first page only
#     page_layout = next(extract_pages(pdf_path))
    
#     rows = {}
#     for element in page_layout:
#         if isinstance(element, LTTextBoxHorizontal):
#             #for text_line in element:
#                 #if isinstance(text_line, LTTextLineHorizontal):
#                     # Check if the text line is within the header area and the left half of the page
#                     if element.y1 > page_layout.height - header_height and element.x0 > page_layout.width / 2:
#                         y_coord = round(element.y1)
#                         if y_coord not in rows:
#                             rows[y_coord] = []
#                         rows[y_coord].append(element.get_text().strip())
                   
                        
    
#     # Sort rows by y-coordinate and concatenate text lines in each row
#     for y_coord in sorted(rows.keys(), reverse=True):
#         header_text.append(' '.join(rows[y_coord]))
    
#     # Print the list of header text
#     return header_text

def header_lister(pdf_path, header_height=100):
    # Extract the layout of the first page only
    page_layout = next(extract_pages(pdf_path))
    
    right_header_text = []
    
    for element in page_layout:
        if isinstance(element, LTTextContainer):
            # Check if the textbox is within the header area and the right half of the page
            if element.y1 > page_layout.height - header_height and element.x0 > page_layout.width / 2:
                # Remove '\n' characters and strip whitespace
                text = element.get_text().replace('\n', ' ').strip()
                right_header_text.append(text)
    
    # Print the text from the right half of the header
    for text in right_header_text:
        print(text)
    
    return text

    


pdf_path = 'Docs 2 pdfs/5370392.pdf'
header_text = header_lister(pdf_path)
print(header_text)

CUST NO
SO
PO
800003388  27219125
INV #
C/NC/CC
159004520 11/11/2020  C     2/10 Pieces




In [719]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer

def extract_right_half_header_text(pdf_path, page_number = 1):
    header_text = []
    for page_layout in extract_pages(pdf_path):
        if page_layout.pageid == page_number:
            text_elements = []
            for element in page_layout:
                if isinstance(element, LTTextContainer):
                    # Check if the element is in the header (top of the page)
                    if element.y1 > page_layout.height - 100:
                        for line in element:
                            if isinstance(line, LTTextContainer):
                                # Check if the line is in the right half of the page
                                if line.x0 > page_layout.width / 2:
                                    text_elements.append(line)
            
            # Sort text elements by y-coordinate (top to bottom) and then by x-coordinate (left to right)
            text_elements.sort(key=lambda e: (-e.y1, e.x0))
            
            # Group text elements by their y-coordinate to ensure row-wise processing
            grouped_elements = {}
            for element in text_elements:
                y_coord = round(element.y1, 1)  # Round to avoid floating-point precision issues
                if y_coord not in grouped_elements:
                    grouped_elements[y_coord] = []
                grouped_elements[y_coord].append(element)
            
            # Process each group of elements
            for y_coord in sorted(grouped_elements.keys(), reverse=True):
                row_text = []
                for element in grouped_elements[y_coord]:
                    row_text.append(element.get_text().strip())
                header_text.append(" ".join(row_text))
    
    return header_text

pdf_path = 'Docs 2 pdfs/5370392.pdf'

header_text = extract_right_half_header_text(pdf_path)
print("Header text from the right half of the page:", header_text)



Header text from the right half of the page: ['800003388', 'CUST NO', '27219125', 'SO', 'PO', '159004520 11/11/2020', 'INV #', 'C     2/10 Pieces', 'C/NC/CC']


In [606]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextBoxHorizontal, LTTextLineHorizontal

def is_table_in_header(pdf_path, header_height=100, min_rows=2, min_columns=2):
    # Extract the layout of the first page only
    page_layout = next(extract_pages(pdf_path))
    
    rows = {}
    for element in page_layout:
        if isinstance(element, LTTextBoxHorizontal):
            for text_line in element:
                if isinstance(text_line, LTTextLineHorizontal):
                    # Check if the text line is within the header area
                    if text_line.y1 > page_layout.height - header_height:
                        y_coord = round(text_line.y1)
                        if y_coord not in rows:
                            rows[y_coord] = []
                        rows[y_coord].append(text_line)
    
    # Check for table-like structure
    row_count = 0
    for y_coord in sorted(rows.keys(), reverse=True):
        if len(rows[y_coord]) >= min_columns:
            row_count += 1
    
    return row_count >= min_rows

# Example usage
pdf_path = 'Docs 2 pdfs/5392919.pdf'
if is_table_in_header(pdf_path):
    print("Table detected in the header.")
else:
    print("No table detected in the header.")


No table detected in the header.


In [1]:
import csv
import os
from pdfminer.high_level import extract_text

def files_iterator(directory):
    csv_file = 'Pranav_RGA_Reports.csv'
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        extracted_text = extract_text(file_path, page_numbers=[0])
        #lines = text.split('\n')  # Split the text into lines
        start_word = 'Customer Problem Description: '
        end_word = 'Material Disposition: '
        start_index = extracted_text.find(start_word)
        end_index = extracted_text.find(end_word, start_index)
        if start_index != -1 and end_index != -1:
            extracted_text = extracted_text[start_index + len(start_word):end_index].strip()
        if start_index != -1 and end_index == -1:
            end_word2 = 'Power Transmission Solutions'
            end_index2 = extracted_text.find(end_word2, start_index)
            extracted_text = extracted_text[start_index + len(start_word):end_index2].strip()
        phrase = "The returned item was inspected"
        phrase_plural = "The returned items were inspected"
        phrase2 = "no manufacturing anomalies"
        phrase3 = "not in compliance with our specifications "
        if phrase2 in extracted_text or phrase3 in extracted_text:
            extracted_text = extracted_text.split(phrase)[0]
            extracted_text = extracted_text.split(phrase_plural)[0]
        
        add_text_to_csv(extracted_text, csv_file)
        
        

# Function to add extracted text to a CSV file
def add_text_to_csv(text, csv_file):
    combined_text = ""
    
    # Try reading the existing content of the CSV file with utf-8 encoding
    try:
        with open(csv_file, 'r', newline='', encoding='utf-8') as file:
            reader = csv.reader(file)
            for row in reader:
                combined_text += ' '.join(row) + ' '
    except UnicodeDecodeError:
        # If utf-8 fails, try reading with latin-1 encoding
        with open(csv_file, 'r', newline='', encoding='latin-1') as file:
            reader = csv.reader(file)
            for row in reader:
                combined_text += ' '.join(row) + ' '
    
    # Append the new text to the combined text
    combined_text += text

    # Write the updated content back to the CSV file with utf-8 encoding
    with open(csv_file, 'w', newline='', encoding='utf-8', errors='replace') as file:
        writer = csv.writer(file)
        writer.writerow([combined_text])


directory = r"Docs 2 pdfs"
files_iterator(directory)


In [17]:
import csv
import os
from pdfminer.high_level import extract_text

# Set a large but manageable field size limit
csv.field_size_limit(10**6)  # 1,000,000 characters

def files_iterator(directory):
    csv_file = 'Pranav_RGA_Reports.csv'
    
    # Clear the CSV file by opening it in write mode
    with open(csv_file, 'w', newline='', encoding='utf-8') as file:
        pass  # This will create an empty file
    
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        extracted_text = extract_text(file_path)
        add_text_to_csv(extracted_text, csv_file)

def add_text_to_csv(text, csv_file):
    combined_text = ""
    
    # Try reading the existing content of the CSV file with utf-8 encoding
    try:
        with open(csv_file, 'r', newline='', encoding='utf-8') as file:
            reader = csv.reader(file)
            for row in reader:
                combined_text += ' '.join(row) + ' '
    except UnicodeDecodeError:
        # If utf-8 fails, try reading with latin-1 encoding
        with open(csv_file, 'r', newline='', encoding='latin-1') as file:
            reader = csv.reader(file)
            for row in reader:
                combined_text += ' '.join(row) + ' '
    
    # Append the new text to the combined text
    combined_text += text
    combined_text += "\n" + "-"*70 + "\n"

    # Check if the combined text exceeds the field size limit
    if len(combined_text) > 10**6:
        # Split the text into chunks that fit within the field size limit
        chunks = [combined_text[i:i + 10**6] for i in range(0, len(combined_text), 10**6)]
    else:
        chunks = [combined_text]

    # Write the chunks to the CSV file with utf-8 encoding
    with open(csv_file, 'w', newline='', encoding='utf-8', errors='replace') as file:
        writer = csv.writer(file)
        for chunk in chunks:
            writer.writerow([chunk])

directory = r"RAG/SampleResumesReal"
files_iterator(directory)


Anna Grimes 
Permanent: 6058 Stirrup Road, Cincinnati, OH 45244 
Local: 2100 S Floyd Street, Louisville, KY 40292 
Phone:  513-404-9864, Email:  amgrim02@louisville.edu 

OBJECTIVE 

First Mechanical Engineering co-op position                                                  January 6th – May 9th, 2025 

EDUCATION 

SKILLS 

Bachelor of Science in Mechanical Engineering                                                          Expected May 2027 
Master of Engineering in Mechanical Engineering                                                     Expected May 2028 
J.B. Speed School of Engineering                                                                                                   GPA 3.6/4.0 
University of Louisville, Kentucky                                                                                 Hours Completed: 78 

Technical Skills/Relevant Coursework 
•  M.E. Dynamics  
•  MS Word, Excel, and PowerPoint 
•  Autodesk Inventor and Revit 
•  Arduino/Python Programmi

In [10]:
from pdfminer.high_level import extract_text

pdf_path = 'RAG/SampleResumesReal/Gieseke, Sarah.pdf'
text = extract_text(pdf_path)
print(text)


In [8]:
import fitz
doc = fitz.open('RAG/SampleResumesReal/Gieseke, Sarah.pdf')
text = ""
for page in doc:
   text+=page.get_text()
print(text+"jjj")


jjj


In [4]:
# importing required modules
from pypdf import PdfReader

# creating a pdf reader object
reader = PdfReader('RAG/SampleResumesReal/Gieseke, Sarah.pdf')

# printing number of pages in pdf file
print(len(reader.pages))

# getting a specific page from the pdf file
page = reader.pages[0]

# extracting text from page
text = page.extract_text()
print(text+"hhh")

1
hhh


In [11]:
import pdfplumber

with pdfplumber.open("RAG/SampleResumesReal/Gieseke, Sarah.pdf") as pdf:
    first_page = pdf.pages[0]  # Access the first page
    text = first_page.extract_text()
    print(text+"hhh")

hhh


In [16]:
import PyPDF2

with open("RAG/ResumeData/data/AVIATION/10176815.pdf", "rb") as file:
    reader = PyPDF2.PdfReader(file)
    page = reader.pages[0]  # Get the first page
    text = page.extract_text()
    print(text+"hhh")

AVIATION ELECTRONICS TECHNICIAN
Summary
Seeking a challenging position where my existing skills and experience contribute to your company while expanding my technical abilities.
Highlights
Proficient with maintenance tracking software, Prezi and Microsoft Office Programs such as Word, Excel, PowerPoint, Outlook, Adobe,
Access, Auto CAD, Commercial Multi-Engine and Single-Engine Pilot, Airline Dispatcher Certified, FAA Class 1 Medical, MATLAB,
labVIEW
Experience
01/2013
 
to 
01/2015
Company Name
Used the instructional design process to plan and develop online courses in partnership with faculty SMEs, then building courses using a
standardized template in the Blackboard LMS environment.
Utilized a variety of technology tools, including Canvas, Blackboard, Respondus, StudyMate, TurnItIn, SnagIt, Camtasia, HTML and
others to create instructional media.
Facilitated in the transfer and development of online courses from Blackboard LMS to Canvas LMS.
Collaborated with remote faculty to prepa